# Score an edit!
In this notebook, we'll look at the the facilities that revscoring gives you for loading in a model and scoring some edits with it.  

## Step 1: Load the scorer model
In this step, we'll load a scorer model file from the disk.

In [1]:
from revscoring import ScorerModel
sm = ScorerModel.load(open("../models/enwiki.damaging.gradient_boosting.model"))
print(sm.format_info())


ScikitLearnClassifier
 - type: GradientBoosting
 - params: balanced_sample_weight=true, random_state=null, loss="deviance", center=true, max_leaf_nodes=null, min_samples_split=2, min_weight_fraction_leaf=0.0, subsample=1.0, learning_rate=0.01, presort="auto", verbose=0, init=null, n_estimators=700, scale=true, balanced_sample=false, max_depth=7, max_features="log2", min_samples_leaf=1, warm_start=false
 - version: 0.3.0
 - trained: 2016-09-27T17:35:33.358346

Table:
	         ~False    ~True
	-----  --------  -------
	False     17177     1690
	True        208      458

Accuracy: 0.903
Precision:
	-----  -----
	False  0.988
	True   0.214
	-----  -----

Recall:
	-----  -----
	False  0.91
	True   0.692
	-----  -----

PR-AUC:
	-----  -----
	False  0.993
	True   0.382
	-----  -----

ROC-AUC:
	-----  -----
	False  0.92
	True   0.921
	-----  -----

Recall @ 0.1 false-positive rate:
	label      threshold    recall    fpr
	-------  -----------  --------  -----
	False          0.032     1      0

## Step 2: Prepare an extractor
We're using a model that was trained on English Wikipedia.  We'll build an extractor to pull features from English Wikipedia. 

In [2]:
import mwapi
from revscoring.extractors import api

extractor = api.Extractor(mwapi.Session("https://en.wikipedia.org", user_agent="Score edit demo in editquality"))

## Step 3: Extract features and score an edit
Now, we'll use the extractor to extract features for a revision ID and then ask the scorer model to generate a prediction based on those features.

In [3]:
feature_values = list(extractor.extract(71076450, sm.features))
list(zip(sm.features, feature_values))[:10]

[(<feature.revision.page.is_articleish>, True),
 (<feature.revision.page.is_mainspace>, True),
 (<feature.revision.page.is_draftspace>, False),
 (<feature.log((wikitext.revision.parent.chars + 1))>, 10.618934240590841),
 (<feature.log((len(<datasource.tokenized(datasource.revision.parent.text)>) + 1))>,
  9.45234493093131),
 (<feature.log((len(<datasource.wikitext.revision.parent.words>) + 1))>,
  8.372860820526318),
 (<feature.log((len(<datasource.wikitext.revision.parent.uppercase_words>) + 1))>,
  5.976350909297934),
 (<feature.log((wikitext.revision.parent.headings + 1))>, 3.091042453358316),
 (<feature.log((wikitext.revision.parent.wikilinks + 1))>, 5.846438775057725),
 (<feature.log((wikitext.revision.parent.external_links + 1))>,
  4.442651256490317)]

In [4]:
sm.score(feature_values)

{'prediction': True,
 'probability': {False: 0.23860839638483888, True: 0.7613916036151611}}